###### init

In [60]:
# import
from elec_bol20 import *
import elec_bol20.util as ebu
import bokeh.layouts
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import Figure
# output_file, show
import bokeh.tile_providers

In [61]:
# p = os.path.pardir(ebu.DIR)

In [62]:
W_CARTO = 500
H_CARTO = 500
ANGLE_DENS = .35
SIZE_DENS = "8pt"
HEIGHT_BDENS = 250
H_FIG_PARTY = 250
BROWSER = 'safari'
MIN_WITH = 600
MAX_WITH = 800
PATH_OUT = 'docs/Ejemplos/z050_panel.html'

COL_LLEGO = '#aaaaaa'
COL_FALTA = '#db2879'

bokeh.plotting.output_file(os.path.join(os.path.dirname(ebu.DIR),
                                        PATH_OUT))
# bokeh.plotting.output_notebook()

In [63]:
df2 = ebu.get_dataframe_2020()

In [64]:
bokeh.plotting.output_notebook()

ll = len(df2)
np.random.seed(100)
df2['xj'] = df2['X'] + np.random.rand(ll) * .5
np.random.seed(200)
df2['yj'] = df2['Y'] + np.random.rand(ll) * .5
cols = ['yj', 'xj', 'PAIS', 'MUN', 'REC', 'HAB', 'COU']

df2['COU'] = 'No'
df2.loc[df2['COUNT'], 'COU'] = 'Sí'

s1 = df2[df2['COUNT']][cols]
s2 = df2[~df2['COUNT']][cols]
sr1 = bokeh.models.ColumnDataSource(s1)
sr2 = bokeh.models.ColumnDataSource(s2)

fig_carto = bokeh.plotting.figure(output_backend="webgl", height=H_CARTO, width=W_CARTO)
fig_carto.scatter(x='xj', y='yj', source=sr2, color=COL_FALTA, radius=.05,
                  alpha=1,
                  legend_label='Mesas por computar (haz click)')
fig_carto.scatter(x='xj', y='yj', source=sr1, color=COL_LLEGO, radius=.05,
                  alpha=1,
                  legend_label='Mesas computadas (haz click)')
fig_carto.legend.click_policy = "hide"

Loading BokehJS ...

In [65]:
dfn = df2.copy()
dfn.loc[~dfn['BOL'], 'DEN_CODES'] = -1

In [66]:
_df = dfn
gr = _df.groupby('DEN_CODES')
den = gr[['HAB']].sum()
den['cum'] = den['HAB'].cumsum()
tot = den['HAB'].sum()
den['mid'] = den['cum'] - den['HAB'] / 2
den['top'] = 100
den['width'] = den['HAB'] - (tot * .01)
den1 = den.copy()

In [67]:
_df = dfn[dfn['COUNT']]
gr = _df.groupby('DEN_CODES')
den = gr[['HAB']].sum()
den['cum'] = den['HAB'].cumsum()
tot = den['HAB'].sum()
den['mid'] = den['cum'] - den['HAB'] / 2
den['top'] = 1
den['width'] = den['HAB'] - (tot * .01)
den2 = den.copy()

In [68]:
dfn['HAB']

ID_MESA
110046138042512901    220
110111036112505502    220
110111036112505503    220
110111036112505504    220
110111036112505505    220
                     ... 
111824437872511201    220
111824437872511203     73
111824637892511401    152
112126638092612401    220
112126638092612402    218
Name: HAB, Length: 35600, dtype: int64

In [69]:
den1['counted'] = den2['HAB'].astype(np.int64)

In [70]:
den1['top_c'] = den1['counted'] / den1['HAB'] * 100

In [71]:
den1.loc[-1, 'mid'] = den1.loc[-1, 'mid'] - 1000000

In [72]:
df2[df2['COUNT']]['HAB'].sum()


def _t(s):
    if np.isnan(s): s = 0
    return f'{s:0.1f}'


def _t1(s):
    if np.isnan(s): s = 0
    return f'{s:0.0f}'


def _t2(r):
    #     return f'{r["tv"]} ({r["tc"]} %)'
    return f'{r["tc"]}%'


den1['tc'] = den1['top_c'].apply(_t)
den1['tv'] = den1['counted'].apply(_t1)
den1['text'] = den1.apply(_t2, axis=1)

In [73]:


fig_bar_dens = bokeh.plotting.figure(height=HEIGHT_BDENS)
# fi = bokeh.plotting.figure()
fig_bar_dens.vbar(x=den1['mid'], width=den1['width'], top=den1['top'],
                  color=COL_FALTA)
fig_bar_dens.vbar(x=den1['mid'], width=den1['width'], top=den1['top_c'],
                  color=COL_LLEGO)
fig_bar_dens.title.text = ''

x = den1[den1.index >= 0]['mid'].mean()
fig_bar_dens.text(x=[x], y=[140], text=['Nacional'])

an = ANGLE_DENS
x = den1[den1.index == 0]['mid'].mean()
fig_bar_dens.text(x=[x], y=[101], text=['Densidad baja (0-50)'], angle=an,
                  text_font_size=SIZE_DENS)

x = den1[den1.index == 1]['mid'].mean()
fig_bar_dens.text(x=[x], y=[101], text=['Densidad media (50-500)'], angle=an,
                  text_font_size=SIZE_DENS)

x = den1[den1.index == 2]['mid'].mean()
fig_bar_dens.text(x=[x], y=[101], text=['Densidad moderada (500-1500)'],
                  angle=an, text_font_size=SIZE_DENS)

x = den1[den1.index == 3]['mid'].mean()
fig_bar_dens.text(x=[x], y=[101], text=['Densidad alta > 1500'], angle=an,
                  text_font_size=SIZE_DENS)

x = den1[den1.index < 0]['mid'].mean()
fig_bar_dens.text(x=[x], y=[110], text=['Exterior'], text_align='center')
fig_bar_dens.xaxis.visible = False
fig_bar_dens.xgrid.visible = False
fig_bar_dens.ygrid.visible = False
fig_carto.yaxis.visible = False
fig_carto.yaxis.visible = False
fig_bar_dens.y_range.start = 0
fig_bar_dens.y_range.end = 160
fig_bar_dens.yaxis.axis_label = "Porcentaje"
ppp = den1['counted'].sum() / den1['HAB'].sum() * 100
fig_bar_dens.title.text = f'Porcentaje de votos computados por densidad (total computado={ppp:0.1f}%)'
fig_bar_dens.xaxis.axis_label = f'Porcentaje total de votos computados = ({ppp:0.1f}%)'
fig_bar_dens.toolbar.logo = None
fig_bar_dens.toolbar_location = None

fig_carto.title.text = "Cartolocación de las mesas"

for l, fig_party in den1.iterrows():
    if l == -1:
        x = fig_party['mid'] + 400000
    else:
        x = fig_party['mid']

    fig_bar_dens.text(x=x, y=[fig_party['tc']], text=[fig_party['text']],
                      text_align='center', text_font_size="8pt",
                      color=[COL_LLEGO])

_c = ['CREEMOS', 'MAS', 'FPV', 'PAN_BOL', 'CC']
dd = df2[[*_c, 'VV']].copy()
res = dd[_c].sum() / dd['VV'].sum() * 100
res.name = 'per'
res.index.name = 'party'
res = res.reset_index()
res['i'] = res.index + .5
se = pd.Series(ebu.C_DIC)
se.name = 'colors'
res = pd.merge(res, se, left_on='party', right_index=True)

source = ColumnDataSource(res)

fig_party = bokeh.plotting.figure(x_range=res['party'], toolbar_location=None,
                                  height=H_FIG_PARTY)
fig_party.vbar(x='i', top='per', width=0.9, source=source,
               line_color='white',
               fill_color=bokeh.transform.factor_cmap('party',
                                                      palette=res['colors'],
                                                      factors=res['party']))


def _f(p): return f'{p:0.1f}%'


res['t'] = res['per'].apply(_f)

fig_party.text(x=res['i'], y=res['per'], text=res['t'], text_align='center')

fig_party.xgrid.grid_line_color = None
fig_party.y_range.start = 0
fig_party.y_range.end = np.ceil(res['per'].max() / 20) * 20
fig_party.title.text =\
    f'Porcentaje sobre el total de votos válidos computados ({ppp:0.1f}%)'

TOOL_TIP = [
    ('Inscritos', '@HAB'),
    ('PAIS, Municipalidad', '@PAIS, @MUN'),
    ('Recinto', '@REC'),
    ('Computada', '@COU'),
    # ('MAS [%]', '@mas{0.0}'),
    # ('CC [%]','@cc{0.0}'),
    # ('Diferencia [%]', '@ad_mas_cc{0.0} (@mas_o_cc)'),
    ('------', '------')
    # ('DEN %', '@DEN')
    # ('PAIS', '@PAIS'),
]

hover_map = bokeh.models.HoverTool(
    tooltips=TOOL_TIP,
    # callback=callback_red_car,
    # renderers = [red_scat_map]
)
fig_carto.add_tools(hover_map)

l0 = bokeh.layouts.column([fig_party, fig_bar_dens, fig_carto],
                          sizing_mode='scale_width')
lay = l0
l0.max_width = MAX_WITH
l0.min_width = MIN_WITH
# lay = bokeh.layouts.row([l0,f])
# lay = p

In [75]:
bokeh.plotting.show(lay, browser=BROWSER)

fatal: pathspec 'docs/Ejemplos/z050_panel.html' did not match any files


In [80]:
!cd {os.path.dirname(ebu.DIR)}; git add {PATH_OUT}; git commit -m'automatic'; git push

[master 890185e] automatic
 1 file changed, 85 insertions(+), 85 deletions(-)
 rewrite docs/Ejemplos/z050_panel.html (68%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 4 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 566.98 KiB | 5.02 MiB/s, done.
Total 5 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/EBol20/elec-bol20.git
   aa1a830..890185e  master -> master


In [82]:
from __future__ import print_function
from datetime import datetime, timedelta
from os import makedirs, walk
from os.path import join, dirname, abspath, sep, exists
import time
import sys
import requests
import json

SLEEP_SECONDS_PER_LOOP = 10 * 60 # feel free to randomize this, lmao.

# sample response to POST: {"fecha":"18/10/2020 18:53:22","correcto":true,"notificacion":"Transacción satisfactoria","datoAdicional":{"nombreArchivo":"exportacion/EG2020_20201018_184655_1350241095801193940.csv","tipoArchivo":"CSV","hash":"d41d8cd98f00b204e9800998ecf8427e","archivo":"https://s3.amazonaws.com/archivo.computo/exportacion/EG2020_20201018_184655_1350241095801193940.csv","fecha":"18/10/2020 06:46:55"}}

# sorry, i straight-up copied these while skimming my own outgoing http requests. i'm an idiot, i apologize. i know most don't matter.
POST_HEADERS = {
    'Accept': "application/json, text/plain, */*",
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9,es;q=0.8,es-419;q=0.7',
    'Captcha': 'nocaptcha',
    'Content-Length': '21',
    'Content-Type': 'application/json',
    'Origin': 'https://computo.oep.org.bo',
    'Referer': 'https://computo.oep.org.bo/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0'
}
GET_HEADERS = {
    'Accept': "*/*",
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9,es;q=0.8,es-419;q=0.7',
    'Connection': 'keep-alive',
    'Host': 's3.amazonaws.com',
    'Origin': 'https://computo.oep.org.bo',
    'Referer': 'https://computo.oep.org.bo/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

POST_PAYLOAD = {'tipoArchivo': "EXCEL"}

# while True:
r = requests.post('https://computo.oep.org.bo/api/v1/descargar', headers=POST_HEADERS, json=POST_PAYLOAD)
print("POST response => {0}".format(r))
print("headers = {0}".format(r.headers))
print("content = {0}".format(r.content))
jresp = json.loads(r.content)
real_url = jresp['datoAdicional']['archivo']
fn = jresp['datoAdicional']['nombreArchivo']
print("making GET request on {0}".format(real_url))

with requests.get(real_url, headers=GET_HEADERS) as r:
    print(f"Writing '{fn}' to file.")
    out_path = "data\\" + fn[:fn.rindex('.')] + str(int(time.time())) + fn[fn.rindex('.'):]
    makedirs(dirname(out_path), exist_ok = True)
    open(out_path, 'wb').write(r.content)

# time.sleep(SLEEP_SECONDS_PER_LOOP) # 10 min.

POST response => <Response [200]>
headers = {'Date': 'Mon, 19 Oct 2020 16:16:00 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=db86d2d51799121133d8edc2b80e2405b1603124160; expires=Wed, 18-Nov-20 16:16:00 GMT; path=/; domain=.oep.org.bo; HttpOnly; SameSite=Lax', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'CF-Cache-Status': 'DYNAMIC', 'cf-request-id': '05e33e7f610000fe506e274000000001', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '5e4bcd123f2bfe50-HEL', 'Content-Encoding': 'gzip'}
content = b'{"fecha":"19/10/2020 12:16:00","correcto":true,"notificacion":"Transacci\xc3\xb3n satisfactoria","datoAdicional":{"nombreArchivo":"exportacion/EG2020_20201019_120834_5458103447920395514.xlsx","tipoArchivo":"EXCEL","hash":"58619ef582ebf524a193a474b090fa65","

KeyboardInterrupt: 

In [84]:
!ls

data\exportacion                z040_explicar_carto_map.ipynb
z010_check_geopadron.ipynb      z040_explicar_carto_map.py
z010_check_geopadron.py         z050_plot_contados.ipynb
z020_generar_densidad_vot.ipynb z050_plot_contados.py
z020_generar_densidad_vot.py    z060_crear_carto_grandeV3.ipynb
z030_generacion_carto.ipynb     z060_crear_carto_grandeV3.py
z030_generacion_carto.py        z061_get_dataframe_2020.py
z037_plots_carto.py
